### copying the precious stuff again.

In [4]:
using Pkg

#Pkg.activate("path/to/project")
Pkg.activate("~/CodeProjects/RosenbluthChains/")
###julia ]; activate ; dev --local PathToRosenbluthPackage
#Pkg.instantiate()
using RosenbluthChains, Plots

  Activating project at `~/Code_Projects/rosenbluthchains/~/CodeProjects/RosenbluthChains`


In [5]:
### type definition
mutable struct My_Measurement{T<:Real} <: AbstractMeasurement 
    x::Vector{T}
    y::Vector{T}
    z::Vector{T}
end
My_Measurement(N::Int) = My_Measurement{Float64}(zeros(N), zeros(N), zeros(N)) ### constructs a Measurement 

### initialise measurments with empty vectors
function RosenbluthChains.InitMeasurement(data::SimData, param::SimulationParameters, Tmp::My_Measurement) 
    return My_Measurement(data.NBeads)
end 


### does nothing
function RosenbluthChains.MeasureAfterBatch(data::SimData, param::SimulationParameters, Measurement::My_Measurement) 
    nothing
end

function RosenbluthChains.MeasureAfterChainGrowth(data::SimData, param::SimulationParameters,Measurement::My_Measurement) 
    Measurement.x = getindex.(data.xyz, 1) ### gets the first element for each element of data.xyz
    Measurement.y = getindex.(data.xyz, 2) ### gets the second element for each element of data.xyz
    Measurement.z = getindex.(data.xyz, 3) ### gets the third element for each element of data.xyz
end

### does nothing
function SaveMeasurements(data::SimData, param::SimulationParameters,Measurement::My_Measurement) 
    nothing
end


#https://hoomd-blue.readthedocs.io/en/v4.7.0/module-md-angle.html#hoomd.md.angle.Harmonic
mutable struct HarmonicBondLength{T<:Real} <: RosenbluthChains.AbstractBondParam 
    k  ::T ### bond strength for all particles
    r0 ::T ### minima of the bond length potential
    Δr ::T ### sampling width=> take values from interval [r0-Δr, r0+Δr]
end
HarmonicBondLength(k ::T, r0::T, Δr::T) where {T<:Real} = HarmonicBondLength{T}(k, r0, Δr) ### constructs a Measurement 

@inline function SetTrialRadius(data::SimData, param::HarmonicBondLength)
   rand!(data.trial_radius, eltype(data.TType)) ### created NTrial many random number ∈ [0,1]
   data.trial_radius .*= 2.0*param.Δr ### ∈ [0, 2Δr]
   data.trial_radius .+= param.r0-param.Δr ### ∈ [r0-Δr, r0+Δr]
   return nothing
end


@inline function GetTrialBoltzmannWeight(data::SimData,param::HarmonicBondLength) 
    fill!(data.tmp4, 0.0) ### temporary array of length NTrials that i will use to store the energy

    data.tmp4  .= @. 1.0/2.0*(data.trial_radius-r0)^2 ### compute energies

    ### here comes a numerical trick to stabilise, we add the energies of al potentials first and take the exponential in "GetTrialBoltzmannWeight" before we have to choose the positions in "ChooseTrialPosition"
    # -= since we have to add all energies and take exp(-kt*E); kt=1
    data.LogBoltzmannFaktor .-= data.tmp4
    return nothing
end

GetTrialBoltzmannWeight (generic function with 1 method)

### Exercise: Write a meausurement for the Radius of Gyration RG and The end-to-end distance REE of a polymer.

### Exercise: Scaling laws of polymers
Many polymer concepts evolve around the concept of scaling laws which deal with the influcence of chain length. The normally have the form of a power law : f(N) = a*N^ν

Can you determine whether RG and REE follow a power law and what these values would be? Use your harmonic bond model and random angles.

How does the influence differ between the an ideal chain (IdealChain()) and a polymer model which uses excluded volumne of radius r (LJ_Repulsion(ones(Int64,N),Dict(1 => r), r ))?

Do the values of RG and REE correlate in any sense?

### Optional Exercise: Persistence length
The persistence length is a measure which determines how quickly the direction of bond vector u_i and u_j become decorrelated so that they are independent of each other. The persistence length is defined as a decay length after which the scalar product <u_i, u_j> = exp(abs(i-j)/lp)=0.5.

Mesure the persistence length for several prefactors of your harmonic angle potential using the following definition of a polymer:
KP = SimulationParameters( FixedBondParameters(r),HarmonicBondAngles(ones(....)), RandTorsion(), LJ_Repulsion(ones(Int64,N),Dict(1 => r), r ))